# 2. Generating a Sample using MS1 Controller

In this notebook, we demonstrate how ViMMS can be used to generate a full-scan mzML file from a single sample. This corresponds to Section 3.1 of the paper.

In [1]:
%matplotlib inline

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('..')

In [4]:
from pathlib import Path

In [5]:
from vimms.Chemicals import ChemicalCreator
from vimms.MassSpec import IndependentMassSpectrometer
from vimms.Controller import SimpleMs1Controller
from vimms.Common import *

Load previously trained spectral feature database and the list of extracted metabolites, created in **01. Download Data.ipynb**.

In [6]:
base_dir = os.path.abspath('example_data')
ps = load_obj(Path(base_dir, 'peak_sampler_mz_rt_int_19_beers_fullscan.p'))
hmdb = load_obj(Path(base_dir, 'hmdb_compounds.p'))

Set ViMMS logging level

In [7]:
set_log_level_debug()

## Create Chemicals

Define an output folder containing our results

In [8]:
out_dir = Path(base_dir, 'results', 'MS1_single')

Here we generate the chemical objects that will be used in the sample. The chemical objects are generated by sampling from metabolites in the HMDB database.

In [9]:
# the list of ROI sources created in the previous notebook '01. Download Data.ipynb'
ROI_Sources = [str(Path(base_dir,'DsDA', 'DsDA_Beer', 'beer_t10_simulator_files'))]

# minimum MS1 intensity of chemicals
min_ms1_intensity = 1.75E5

# m/z and RT range of chemicals
rt_range = [(0, 1440)]
mz_range = [(0, 1050)]

# the number of chemicals in the sample
n_chems = 6500

# maximum MS level (we do not generate fragmentation peaks when this value is 1)
ms_level = 1

In [10]:
chems = ChemicalCreator(ps, ROI_Sources, hmdb)
dataset = chems.sample(mz_range, rt_range, min_ms1_intensity, n_chems, ms_level)
save_obj(dataset, Path(out_dir, 'dataset.p'))

DEBUG  : ChemicalCreator                : Sorting database compounds by masses
DEBUG  : ChemicalCreator                : 6500 chemicals to be created.
DEBUG  : ChemicalCreator                : Sampling formula 0/6500
DEBUG  : ChemicalCreator                : Sampling formula 500/6500
DEBUG  : ChemicalCreator                : Sampling formula 1000/6500
DEBUG  : ChemicalCreator                : Sampling formula 1500/6500
DEBUG  : ChemicalCreator                : Sampling formula 2000/6500
DEBUG  : ChemicalCreator                : Sampling formula 2500/6500
DEBUG  : ChemicalCreator                : Sampling formula 3000/6500
DEBUG  : ChemicalCreator                : Sampling formula 3500/6500
DEBUG  : ChemicalCreator                : Sampling formula 4000/6500
DEBUG  : ChemicalCreator                : Sampling formula 4500/6500
DEBUG  : ChemicalCreator                : Sampling formula 5000/6500
DEBUG  : ChemicalCreator                : Sampling formula 5500/6500
DEBUG  : ChemicalCreator 

Created /home/cjurich/projects/vimms/examples/example_data/results/MS1_single
Saving <class 'list'> to /home/cjurich/projects/vimms/examples/example_data/results/MS1_single/dataset.p


In [11]:
for chem in dataset[0:10]:
    print(chem)

KnownChemical - 'C13H12O3' rt=787.87 max_intensity=23718771.78
KnownChemical - 'C9H14N2O3S2' rt=224.35 max_intensity=6824500.18
KnownChemical - 'C19H12O2' rt=894.10 max_intensity=195695.90
KnownChemical - 'C16H19NO10' rt=785.77 max_intensity=1533328.27
KnownChemical - 'C3H5O7P' rt=204.98 max_intensity=737461.08
KnownChemical - 'C16H10N2O8S2' rt=538.08 max_intensity=21184668.53
KnownChemical - 'C10H17N3O8' rt=610.93 max_intensity=1315789.26
KnownChemical - 'C29H40O8' rt=537.58 max_intensity=1119299.33
KnownChemical - 'C19H16O9' rt=333.55 max_intensity=251170.17
KnownChemical - 'C23H48NO7P' rt=325.95 max_intensity=260464.78


## Run MS1 controller on the samples and generate .mzML files

In [12]:
set_log_level_warning()

In [13]:
min_rt = rt_range[0][0]
max_rt = rt_range[0][1]

In [14]:
mass_spec = IndependentMassSpectrometer(POSITIVE, dataset, ps)
controller = SimpleMs1Controller(mass_spec)
controller.run(min_rt, max_rt)

mzml_filename = Path(out_dir, 'ms1_controller.mzML')
controller.write_mzML('my_analysis', mzml_filename)

(1440.580s) ms_level=1: : 1440.5803710000002it [00:59, 24.09it/s]                         


Simulated results have been saved to the following .mzML file and can be viewed in tools like [ToppView](https://pubs.acs.org/doi/abs/10.1021/pr900171m) or using other mzML file viewers.

In [15]:
mzml_filename

PosixPath('/home/cjurich/projects/vimms/examples/example_data/results/MS1_single/ms1_controller.mzML')